In [1]:
# Install required packages.
import os

!pip install torch==1.12.1+cu113 torchvision==0.13.1+cu113 torchaudio==0.12.1 --extra-index-url https://download.pytorch.org/whl/cu113
import torch
os.environ['TORCH'] = torch.__version__
print(torch.__version__)

!pip install -q torch-scatter -f https://data.pyg.org/whl/torch-${TORCH}.html
!pip install -q torch-sparse -f https://data.pyg.org/whl/torch-${TORCH}.html
!pip install -q git+https://github.com/pyg-team/pytorch_geometric.git

# Helper function for visualization.
%matplotlib inline
import networkx as nx
import matplotlib.pyplot as plt


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/, https://download.pytorch.org/whl/cu113
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╸ 1.8/1.8 GB 85.6 MB/s eta 0:00:01tcmalloc: large alloc 1837744128 bytes == 0x38f0000 @  0x7ff4361be1e7 0x4d30a0 0x4d312c 0x5d6f4c 0x51edd1 0x51ef5b 0x4f750a 0x4997a2 0x55cd91 0x5d8941 0x4997a2 0x55cd91 0x5d8941 0x4997a2 0x55cd91 0x5d8941 0x4997a2 0x55cd91 0x5d8941 0x4997a2 0x55cd91 0x5d8941 0x4997a2 0x5d8868 0x4997a2 0x55cd91 0x5d8941 0x49abe4 0x55cd91 0x5d8941 0x4997a2
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╸ 1.8/1.8 GB 147.2 MB/s eta 0:00:01tcmalloc: large alloc 2297184256 bytes == 0x7118c000 @  0x7ff4361bf615 0x5d6f4c 0x51edd1 0x51ef5b 0x4f750a 0x4997a2 0x55cd91 0x5d8941 0x4997a2 0x55cd91 0x5d8941 0x4997a2 0x55cd91 0x5d8941 0x4997a2 0x55cd91 0x5d8941 0x4997a2 0x55cd91 0x5d8941 0x4997a2 0x5d8868 0x4997a2 0x55cd91 0x5d8941 0x49abe4 0x55cd91 0x5d8941 0x4997a2 0x55cd91 0x5d8941
     ━━━━━━━━━━━━━━━━━━

In [2]:
pip install deepchem==2.6.1

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 608.3/608.3 KB 31.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 29.3/29.3 MB 49.7 MB/s eta 0:00:00


In [3]:
pip install hyperopt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [4]:
import torch
import pandas as pd
import numpy as np
import torch_geometric.transforms as T
import torch_geometric
from torch_geometric.data import Dataset
import numpy as np 
import os
import deepchem as dc
import torch.nn as nn
from torch.nn import Linear,BatchNorm1d
from torch_geometric.nn import GCNConv,GATv2Conv
from torch_geometric.nn import BatchNorm
from tqdm import tqdm
from torch_geometric.data import DataLoader
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
from sklearn.metrics import precision_recall_curve
from matplotlib import pyplot
from sklearn import metrics
from sklearn.preprocessing import label_binarize
from sklearn.metrics import average_precision_score
import seaborn as sns
import pandas as pd 
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix
import math
from sklearn.metrics import accuracy_score
import warnings
warnings.filterwarnings("ignore")

In [5]:
data = pd.read_csv('raw/totalData.csv')
data.head()

,Unnamed: 0,Drug1_ID,Drug1_SMILES
0,0,DB00706,CCOc1ccccc1OCCN[C@H](C)Cc1ccc(OC)c(S(N)(=O)=O)c1
1,1,DB01023,CCOC(=O)C1=C(C)NC(C)=C(C(=O)OC)C1c1cccc(Cl)c1Cl
2,2,3849.0,C[C@@H](Oc1ccc2[nH]nc(/C=C/c3cnn(CCO)c3)c2c1)c...
3,3,3515.0,COc1ccc2c(c1)c(CC(=O)O)c(C)n2C(=O)c1ccc(I)cc1
4,4,6380.0,O=P(O)(O)OCCCc1c[nH]c2ccccc12


In [6]:
len(data)

10951

In [8]:
remove = []

class MoleculeDataset(Dataset):
    def __init__(self, root, filename, test=False, transform=None, pre_transform=None):
        """
        root = Where the dataset should be stored. This folder is split
        into raw_dir (downloaded dataset) and processed_dir (processed data). 
        """
        self.test = test
        self.filename = filename
        self.sz = 0
        super(MoleculeDataset, self).__init__(root, transform, pre_transform)
        
    @property
    def raw_file_names(self):
        """ If this file exists in raw_dir, the download is not triggered.
            (The download func. is not implemented here)  
        """
        return self.filename

    @property
    def processed_file_names(self):
        """ If these files are found in raw_dir, processing is skipped"""
        self.data = pd.read_csv(self.raw_paths[0]).reset_index()

        if self.test:
            return [f'data_test_{i}.pt' for i in list(self.data.index)]
        else:
            return [f'data_{i}.pt' for i in list(self.data.index)]
        

    def download(self):
        pass

    def process(self):
        self.data = pd.read_csv(self.raw_paths[0]).reset_index()
        featurizer = dc.feat.MolGraphConvFeaturizer(use_edges=True)
        count = 0
        for index, mol in tqdm(self.data.iterrows(), total=self.data.shape[0]):
            # Featurize molecule
            if type(mol["Drug1_SMILES"])==float:
              remove.append(mol["Drug1_ID"])
              continue
            f = featurizer.featurize(mol["Drug1_SMILES"])
            if not f : 
              remove.append(mol["Drug1_ID"])
              print(count)
              continue
            data = f[0].to_pyg_graph()
            data.y = 1
            data.smiles = mol["Drug1_SMILES"]
            data.id = mol["Drug1_ID"]
            if self.test:
                torch.save(data, 
                    os.path.join(self.processed_dir, 
                                f'data_test_{count}.pt'))
            else:
                torch.save(data, 
                    os.path.join(self.processed_dir, 
                                f'data_{count}.pt'))
            count+=1
        self.sz = count

    def _get_label(self, label):
        label = np.asarray([label])
        return torch.tensor(label, dtype=torch.int64)

    def len(self):
        return self.sz
        #return self.data.shape[0]

    def get(self, idx):
        """ - Equivalent to __getitem__ in pytorch
            - Is not needed for PyG's InMemoryDataset
        """
        if self.test:
            data = torch.load(os.path.join(self.processed_dir, 
                                 f'data_test_{idx}.pt'))
        else:
            data = torch.load(os.path.join(self.processed_dir, 
                                 f'data_{idx}.pt'))        
        return data

encoder_set = MoleculeDataset("","totalData.csv")

Processing...
  1%|          | 80/10951 [00:00<01:55, 94.33it/s] 

68
76


  2%|▏         | 169/10951 [00:02<02:06, 85.55it/s]

149
163


  3%|▎         | 371/10951 [00:05<01:53, 93.34it/s] 

...

100%|█████████▉| 10912/10951 [02:39<00:00, 75.45it/s]

10778
10782


100%|█████████▉| 10921/10951 [02:39<00:00, 49.20it/s]

10789


100%|██████████| 10951/10951 [02:41<00:00, 67.95it/s]
Done!


In [7]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cuda', index=0)

In [9]:
len(encoder_set)

10820

In [22]:
class GVAE(nn.Module):
    def __init__(self, encoder_neurons, latent_size, decoder_neurons, decoder_size):
        super(GVAE, self).__init__()
        feature_size=encoder_set[0].x.shape[1]
        self.encoder_embedding_size = encoder_neurons
        edge_dim = 11
        self.latent_embedding_size = latent_size
        self.decoder_hidden_neurons = decoder_neurons
        decoder_size = decoder_size

        # Encoder layers
        # self.conv1 = GCNConv(feature_size, self.encoder_embedding_size)
        self.conv1 = GATv2Conv(feature_size, self.encoder_embedding_size,
                             edge_dim=edge_dim, heads=4, concat=False, beta=True)
        self.bn1 = BatchNorm(self.encoder_embedding_size)
        self.conv2 = GATv2Conv(self.encoder_embedding_size, self.encoder_embedding_size,
                             edge_dim=edge_dim, heads=4, concat=False, beta=True)
        self.bn2 = BatchNorm(self.encoder_embedding_size)
        self.conv3 = GATv2Conv(self.encoder_embedding_size, self.encoder_embedding_size,
                             edge_dim=edge_dim, heads=4, concat=False, beta=True)
        self.bn3 = BatchNorm(self.encoder_embedding_size)
        

        # Latent transform layers
        self.mu_transform = GATv2Conv(self.encoder_embedding_size, self.latent_embedding_size, 
                                   heads=4, concat=False, beta=True, edge_dim=edge_dim)
        self.logvar_transform = GATv2Conv(self.encoder_embedding_size, self.latent_embedding_size, 
                                       heads=4, concat=False, beta=True, edge_dim=edge_dim)
        

        
        self.decoder_dense_2 = Linear(self.latent_embedding_size*2, decoder_size)
        self.decoder_bn_2 = BatchNorm1d(decoder_size)
        self.decoder_dense_3 = Linear(self.latent_embedding_size*2, decoder_size)
        self.decoder_bn_3 = BatchNorm1d(decoder_size)
        self.decoder_dense_4 = Linear(decoder_size, 1)
        

    def encode(self, x, edge_attr, edge_index, batch_index):
        # GNN layers
        x = self.conv1(x, edge_index, edge_attr).relu()
        # x = self.conv1(x, edge_index, edge_attr).relu()
        x = self.bn1(x)
        x = self.conv2(x, edge_index, edge_attr).relu()
        x = self.bn2(x)
        x = self.conv3(x, edge_index, edge_attr).relu()
        x = self.bn3(x)
        
        # Latent transform layers
        mu = self.mu_transform(x, edge_index, edge_attr)
        logvar = self.logvar_transform(x, edge_index, edge_attr)
        return mu, logvar


    def decode(self, z, batch_index):
        """
        Takes n latent vectors (one per node) and decodes them
        into the upper triangular part of their adjacency matrix.
        """
        inputs = []

        # Iterate over molecules in batch
        for graph_id in torch.unique(batch_index):
            graph_mask = torch.eq(batch_index, graph_id)
            graph_z = z[graph_mask]

            # Get indices for triangular upper part of adjacency matrix
            # graph_z.shape[0] = number of nodes | triu_indices()=gives edge_indices for the graph
            edge_indices = torch.triu_indices(graph_z.shape[0], graph_z.shape[0], offset=1)

            # Repeat indices to match dim of latent codes
            dim = self.latent_embedding_size
            source_indices = torch.reshape(edge_indices[0].repeat_interleave(dim), (edge_indices.shape[1], dim))
            target_indices = torch.reshape(edge_indices[1].repeat_interleave(dim), (edge_indices.shape[1], dim))

            # Gather features=sampled latent vectors
            sources_feats = torch.gather(graph_z, 0, source_indices.to(device))
            target_feats = torch.gather(graph_z, 0, target_indices.to(device))

            # Concatenate inputs of all source and target nodes
            # graph_input shape = [no of pairs of nodes, dimension of each pair]
            graph_inputs = torch.cat([sources_feats, target_feats], axis=1)
            inputs.append(graph_inputs)

        # Concatenate all inputs of all graphs in the batch
        inputs = torch.cat(inputs)

        x = self.decoder_dense_2(inputs).relu()
        x = self.decoder_bn_2(x)
        x = self.decoder_dense_3(inputs).relu()
        x = self.decoder_bn_3(x)
        edge_logits = self.decoder_dense_4(x)
   

        return edge_logits


    def reparameterize(self, mu, logvar):
        """
        The reparametrization trick is required to 
        backpropagate through the network.
        We cannot backpropagate through a "sampled"
        node as it is not deterministic.
        The trick is to separate the randomness
        from the network.
        """
        if self.training:
            # Get standard deviation
            std = torch.exp(logvar)
            # Returns random numbers from a normal distribution
            eps = torch.randn_like(std)
            # Return sampled values
            return eps.mul(std).add_(mu)
        else:
            return mu

    def forward(self, x, edge_attr, edge_index, batch_index):
        # Encode the molecule
        mu, logvar = self.encode(x, edge_attr, edge_index, batch_index)
        # Sample latent vector (per atom)
        z = self.reparameterize(mu, logvar)
        # Decode latent vector into original molecule
        triu_logits = self.decode(z, batch_index) #output=triangular upper part of the matrix

        return triu_logits, mu, logvar

    
   
# model = GVAE(feature_size=encoder_set[0].x.shape[1], encoder_neurons, decoder_neurons, decoder_size)
# model = GVAE(encoder_neurons, latent_size, decoder_neurons, decoder_size)
# print(model)
# print("Number of parameters: ", sum(p.numel() for p in model.parameters()))

In [23]:
from torch_geometric.data import DataLoader

data_size = len(encoder_set)
NUM_GRAPHS_PER_BATCH = 64
frac = 0.5
train_loader = DataLoader(encoder_set[:int(data_size * frac)], 
                    batch_size=NUM_GRAPHS_PER_BATCH, shuffle=True)
test_loader = DataLoader(encoder_set[int(data_size * frac):], 
                         batch_size=NUM_GRAPHS_PER_BATCH, shuffle=True)


# model = model.to(device)


In [24]:
from torch_geometric.utils import to_dense_adj

def kl_loss(mu=None, logstd=None):
    """
    Closed formula of the KL divergence for normal distributions
    """
    MAX_LOGSTD = 10
    logstd =  logstd.clamp(max=MAX_LOGSTD)
    kl_div = -0.5 * torch.mean(torch.sum(1 + 2 * logstd - mu**2 - logstd.exp()**2, dim=1))

    # Limit numeric errors
    kl_div = kl_div.clamp(max=1000)
    return kl_div

def slice_graph_targets(graph_id, batch_targets, batch_index):
    # Create mask for nodes of this graph id
    graph_mask = torch.eq(batch_index, graph_id)
    # Row slice and column slice batch targets to get graph targets
    graph_targets = batch_targets[graph_mask][:, graph_mask]
    # Get triangular upper part of adjacency matrix for targets
    triu_indices = torch.triu_indices(graph_targets.shape[0], graph_targets.shape[0], offset=1)
    triu_mask = torch.squeeze(to_dense_adj(triu_indices)).bool()
    return graph_targets[triu_mask]

def slice_graph_predictions(triu_logits, graph_triu_size, start_point):

    graph_logits_triu = torch.squeeze(triu_logits[start_point:start_point + graph_triu_size])  
    return graph_logits_triu

def slice_node_features(graph_id, node_features, batch_index):
    # Create mask for nodes of this graph id
    graph_mask = torch.eq(batch_index, graph_id)
    graph_node_features = node_features[graph_mask]
    return graph_node_features

  

def gvae_loss(triu_logits, edge_index, mu, logvar, batch_index, kl_beta):
    """
    Calculates a weighted ELBO loss for a batch of graphs for the graph
    variational autoencoder model.
    """
    # Convert target edge index to dense adjacency matrix
    batch_targets = torch.squeeze(to_dense_adj(edge_index))

    # Reconstruction loss per graph
    batch_recon_loss = []
    batch_node_counter = 0

    # Loop over graphs in this batch
    for graph_id in torch.unique(batch_index):
        # Get upper triangular targets for this graph from the whole batch
        graph_targets_triu = slice_graph_targets(graph_id, 
                                                batch_targets, 
                                                batch_index)

        # Get upper triangular predictions for this graph from the whole batch
        graph_predictions_triu = slice_graph_predictions(triu_logits, 
                                                        graph_targets_triu.shape[0], 
                                                        batch_node_counter)
        
        # Update counter to the index of the next graph
        batch_node_counter = batch_node_counter + graph_targets_triu.shape[0]

        # Calculate edge-weighted binary cross entropy
        weight = graph_targets_triu.shape[0]/sum(graph_targets_triu)
        bce = torch.nn.BCEWithLogitsLoss(pos_weight=weight).to(device)
        graph_recon_loss = bce(graph_predictions_triu.view(-1), graph_targets_triu.view(-1))
        batch_recon_loss.append(graph_recon_loss)   

    # Take average of all losses
    num_graphs = torch.unique(batch_index).shape[0]
    batch_recon_loss = sum(batch_recon_loss) / num_graphs
    
    # KL Divergence
    kl_divergence = kl_loss(mu, logvar)

    return batch_recon_loss + kl_beta * kl_divergence, kl_divergence


def reconstruction_accuracy(triu_logits, edge_index, batch_index, node_features):
    # Convert edge index to adjacency matrix
    batch_targets = torch.squeeze(to_dense_adj(edge_index))
    # Store target trius
    batch_targets_triu = []
    # Iterate over batch and collect each of the trius
    batch_node_counter = 0
    num_recon = 0
    for graph_id in torch.unique(batch_index):
        # Get triu parts for this graph
        graph_targets_triu = slice_graph_targets(graph_id, 
                                                batch_targets, 
                                                batch_index)
        graph_predictions_triu = slice_graph_predictions(triu_logits, 
                                                        graph_targets_triu.shape[0], 
                                                        batch_node_counter)

        # Update counter to the index of the next graph
        batch_node_counter = batch_node_counter + graph_targets_triu.shape[0]

        # Slice node features of this batch
        graph_node_features = slice_node_features(graph_id, node_features, batch_index)

        # Check if graph is successfully reconstructed
        num_nodes = sum(torch.eq(batch_index, graph_id))
      
        batch_targets_triu.append(graph_targets_triu)
        
    batch_targets_triu = torch.cat(batch_targets_triu).detach()
    triu_discrete = torch.squeeze(torch.tensor(torch.sigmoid(triu_logits) > 0.5, dtype=torch.int32))
    acc = torch.true_divide(torch.sum(batch_targets_triu==triu_discrete), batch_targets_triu.shape[0]) 
    return acc.detach().cpu().numpy(), num_recon
    




In [25]:
loss_fn = gvae_loss
# optimizer = torch.optim.Adam(model.parameters(), lr=0.002)
kl_beta = 0.005

In [26]:
def run_one_epoch(data_loader, model, type, epoch, kl_beta):
    # Store per batch loss and accuracy 
    all_losses = []
    all_accs = []
    optimizer = torch.optim.Adam(model.parameters(), lr=0.002)

    # Iterate over data loader
    for i, batch in enumerate(tqdm(data_loader)):
        # Some of the data points have invalid adjacency matrices 
        try:
            # Use GPU
            batch.to(device)  
            # Reset gradients
            optimizer.zero_grad() 
            # Call model
            triu_logits, mu, logvar = model(batch.x.float(), 
                                            batch.edge_attr.float(),
                                            batch.edge_index, 
                                            batch.batch) 
            # Calculate loss and backpropagate
            loss, kl_div = loss_fn(triu_logits, batch.edge_index, mu, logvar, batch.batch, kl_beta)
            if type == "Train":
                loss.backward()  
                optimizer.step()  
            # Calculate metrics
            acc = reconstruction_accuracy(triu_logits, batch.edge_index, batch.batch, batch.x.float())
            

            # Store loss and metrics
            all_losses.append(loss.detach().cpu().numpy())
            all_accs.append(acc)
            
            

        except IndexError as error:
            # For a few graphs the edge information is not correct
            # Simply skip the batch containing those
            print("Error: ", error) 
    
    print(f"{type} epoch {epoch} loss: ", np.array(all_losses).mean())
    print(f"{type} epoch {epoch} accuracy: ", np.array(all_accs).mean())
    return np.array(all_losses).mean()
   

In [27]:
from hyperopt import partial
from hyperopt import hp, fmin, tpe, Trials

In [28]:
from hyperopt.pyll.base import scope

In [29]:
def optimize(params, train_loader, test_loader):
  model = GVAE(**params)
  model.to(device)
  fold_loss = []
  for epoch in range(3): 
    model.train()
    loss1 = run_one_epoch(train_loader, model, type="Train", epoch=epoch, kl_beta=kl_beta)
    fold_loss.append(loss1)
    if epoch % 5 == 0:
        print("test epoch...")
        model.eval()
        loss2 = run_one_epoch(test_loader, model, type="Test", epoch=epoch, kl_beta=kl_beta)
        fold_loss.append(loss2)
  return np.mean(fold_loss)


In [30]:
param_space = {
    "encoder_neurons" : scope.int(hp.quniform("encoder_neurons", 50, 150,1)),
    "latent_size" : scope.int(hp.quniform("latent_size", 8, 12,1)),
    "decoder_neurons" : scope.int(hp.quniform("decoder_neurons", 50, 150,1)),
    "decoder_size" : scope.int(hp.quniform("decoder_size", 50, 150,1))
}

optimization_function = partial(optimize, train_loader=train_loader, test_loader=test_loader)

trials = Trials()

result = fmin(
    fn = optimization_function,
    space = param_space,
    algo = tpe.suggest,
    max_evals = 6,
    trials = trials
)

print(result)

  0%|          | 0/6 [00:00<?, ?it/s, best loss: ?]

  0%|          | 0/85 [00:00<?, ?it/s]

  1%|1         | 1/85 [00:01<02:34,  1.84s/it]

  2%|2         | 2/85 [00:03<02:10,  1.57s/it]

  4%|3         | 3/85 [00:04<02:03,  1.50s/it]

  5%|4         | 4/85 [00:06<02:01,  1.50s/it]

  6%|5         | 5/85 [00:07<01:50,  1.38s/it]

  7%|7         | 6/85 [00:08<01:50,  1.39s/it]

  8%|8         | 7/85 [00:09<01:45,  1.35s/it]

  9%|9         | 8/85 [00:11<01:44,  1.36s/it]

 11%|#         | 9/85 [00:12<01:37,  1.28s/it]

 12%|#1        | 10/85 [00:14<01:52,  1.49s/it]

 13%|#2        | 11/85 [00:15<01:43,  1.40s/it]

 14%|#4        | 12/85 [00:17<01:44,  1.43s/it]

 15%|#5        | 13/85 [00:18<01:40,  1.39s/it]

 16%|#6        | 14/85 [00:19<01:34,  1.34s/it]

 18%|#7        | 15/85 [00:20<01:32,  1.32s/it]

 19%|#8        | 16/85 [00:22<01:34,  1.37s/it]

 20%|##        | 17/85 [00:23<01:32,  1.36s/it]

 21%|##1       | 18/85 [00:25<01:32,  1.39s/it]

 22%|##2       | 19/85 [00:26<01:33,  1.41s/it]

 24%|##3       | 20/85 [00:27<01:28,  

Train epoch 0 loss: 
1.0942727
Train epoch 0 accuracy: 
0.26990702099659863
test epoch...
  0%|          | 0/6 [01:56<?, ?it/s, best loss: ?]

  0%|          | 0/85 [00:00<?, ?it/s]

  1%|1         | 1/85 [00:01<01:52,  1.34s/it]

  2%|2         | 2/85 [00:02<01:53,  1.36s/it]

  4%|3         | 3/85 [00:03<01:45,  1.29s/it]

  5%|4         | 4/85 [00:05<01:49,  1.35s/it]

  6%|5         | 5/85 [00:06<01:41,  1.27s/it]

  7%|7         | 6/85 [00:08<01:46,  1.35s/it]

  8%|8         | 7/85 [00:09<01:49,  1.41s/it]

  9%|9         | 8/85 [00:10<01:46,  1.39s/it]

 11%|#         | 9/85 [00:14<02:30,  1.98s/it]

 12%|#1        | 10/85 [00:15<02:13,  1.78s/it]

 13%|#2        | 11/85 [00:17<02:20,  1.90s/it]

 14%|#4        | 12/85 [00:19<02:13,  1.83s/it]

 15%|#5        | 13/85 [00:21<02:28,  2.06s/it]

 16%|#6        | 14/85 [00:23<02:10,  1.84s/it]

 18%|#7        | 15/85 [00:25<02:22,  2.03s/it]

 19%|#8        | 16/85 [00:26<02:03,  1.79s/it]

 20%|##        | 17/85 [00:28<01:49,  1.61s/it]

 21%|##1       | 18/85 [00:29<01:36,  1.44s/it]

 22%|##2       | 19/85 [00:30<01:35,  1.45s/it]

 24%|##3       | 20/85 [00:31<01:31,  

Test epoch 0 loss: 
0.88571197
Test epoch 0 accuracy: 
0.30295863870312184
  0%|          | 0/6 [03:57<?, ?it/s, best loss: ?]

...

Test epoch 0 loss: 
0.8450099
Test epoch 0 accuracy: 
0.3115961285198436
 83%|████████▎ | 5/6 [43:05<07:49, 469.16s/it, best loss: 0.828944206237793]

  0%|          | 0/85 [00:00<?, ?it/s]

  1%|1         | 1/85 [00:01<01:46,  1.27s/it]

  2%|2         | 2/85 [00:02<01:44,  1.26s/it]

  4%|3         | 3/85 [00:04<01:52,  1.37s/it]

  5%|4         | 4/85 [00:05<01:43,  1.27s/it]

  6%|5         | 5/85 [00:06<01:43,  1.29s/it]

  7%|7         | 6/85 [00:07<01:37,  1.24s/it]

  8%|8         | 7/85 [00:08<01:34,  1.22s/it]

  9%|9         | 8/85 [00:10<01:37,  1.27s/it]

 11%|#         | 9/85 [00:12<01:54,  1.51s/it]

 12%|#1        | 10/85 [00:13<01:48,  1.45s/it]

 13%|#2        | 11/85 [00:14<01:41,  1.38s/it]

 14%|#4        | 12/85 [00:15<01:37,  1.34s/it]

 15%|#5        | 13/85 [00:17<01:33,  1.30s/it]

 16%|#6        | 14/85 [00:18<01:33,  1.32s/it]

 18%|#7        | 15/85 [00:19<01:29,  1.28s/it]

 19%|#8        | 16/85 [00:21<01:30,  1.31s/it]

 20%|##        | 17/85 [00:22<01:34,  1.38s/it]

 21%|##1       | 18/85 [00:23<01:30,  1.35s/it]

 22%|##2       | 19/85 [00:25<01:25,  1.30s/it]

 24%|##3       | 20/85 [00:26<01:23,  

Train epoch 1 loss: 
0.7107242
Train epoch 1 accuracy: 
0.3846119084778954
 83%|████████▎ | 5/6 [44:59<07:49, 469.16s/it, best loss: 0.828944206237793]

  0%|          | 0/85 [00:00<?, ?it/s]

  1%|1         | 1/85 [00:01<01:36,  1.15s/it]

  2%|2         | 2/85 [00:03<02:21,  1.71s/it]

  4%|3         | 3/85 [00:05<02:30,  1.83s/it]

  5%|4         | 4/85 [00:06<02:09,  1.59s/it]

  6%|5         | 5/85 [00:07<02:00,  1.50s/it]

  7%|7         | 6/85 [00:09<02:11,  1.67s/it]

  8%|8         | 7/85 [00:11<02:03,  1.58s/it]

  9%|9         | 8/85 [00:12<01:53,  1.47s/it]

 11%|#         | 9/85 [00:13<01:46,  1.41s/it]

 12%|#1        | 10/85 [00:15<01:43,  1.38s/it]

 13%|#2        | 11/85 [00:16<01:43,  1.40s/it]

 14%|#4        | 12/85 [00:17<01:35,  1.31s/it]

 15%|#5        | 13/85 [00:18<01:33,  1.29s/it]

 16%|#6        | 14/85 [00:20<01:38,  1.39s/it]

 18%|#7        | 15/85 [00:21<01:31,  1.31s/it]

 19%|#8        | 16/85 [00:23<01:38,  1.43s/it]

 20%|##        | 17/85 [00:25<01:53,  1.67s/it]

 21%|##1       | 18/85 [00:26<01:43,  1.54s/it]

 22%|##2       | 19/85 [00:27<01:36,  1.46s/it]

 24%|##3       | 20/85 [00:29<01:34,  

Train epoch 2 loss: 
0.55764955
Train epoch 2 accuracy: 
0.4247726300183465
100%|██████████| 6/6 [46:57<00:00, 469.50s/it, best loss: 0.7951616048812866]
{'decoder_neurons': 137.0, 'decoder_size': 113.0, 'encoder_neurons': 119.0, 'latent_size': 9.0}
